### Train notebooks

In [ ]:
import os
import sys
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(stream=sys.stdout, format='',
                level=logging.INFO, datefmt=None)
logger = logging.getLogger('RCD-train')

In [ ]:
#https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False 

import os
import sys
from pathlib import Path
import json

import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import PIL
from PIL import Image, ImageDraw
from io import BytesIO
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (8.0, 6.0)
plt.rcParams['axes.titlepad'] = 20 

from IPython.display import display, Markdown, HTML

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import utils
import importlib
importlib.reload(utils)

In [ ]:
# https://arxiv.org/abs/2104.00298
model_name = "efficientnetv2-xl-21k"
model_weight_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2"
IMAGE_SIZE = (512, 512)
PATH_DATA_DIR = "../dataset/sih_road_dataset"
BATCH_SIZE = 16

In [ ]:
train_ds = utils.build_dataset(data_dir=PATH_DATA_DIR, subset="training", img_size=IMAGE_SIZE)
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

normalization_layer = tf.keras.layers.Rescaling(1. / 255)

preprocessing_model = utils.get_augmentation_model(normalization_layer)
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = utils.build_dataset(data_dir=PATH_DATA_DIR, subset="validation", img_size=IMAGE_SIZE)
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

logger.info(f'Train size: {train_size} \nValidation size: {valid_size}')

In [ ]:
logger.info(f"Building model with {model_weight_url}")
do_fine_tuning = False
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(model_weight_url, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(class_names),
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [ ]:
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE
hist = model.fit(
    train_ds,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps).history